## MiniCaml con Insiemi

Vogliamo estendere l'interprete visto a lezione, implementando un costrutto che permetta di formare **insiemi di elementi dello stesso tipo**. Un insieme è una sequenza di elementi **senza duplicati**, e può contenere unicamente **interi, booleani e stringhe**. Esempi di espressioni in questo linguaggio esteso potrebbero essere:
```
{int: 1, 2, 3, 4};;

{int:};;

{int: 4};;

let A = {int: 1, 2, 3, 4} in 
    insert(6, A);;

let A = {string: "hello", "world", "from", "MiniCaml" } in
    if ("hi" in A) then 0 else 1

let A = {string: "hello", "world", "from", "MiniCaml" } in
let B = {string: "hi" "world"} in
    intersection A B
```

### Es 1: Albero della sintassi astratta

Estendere l'AST di MiniCaml con un **nuovo tipo** e le seguenti **nuove espressioni**:
1. Espressioni per **costruire insiemi** (composti unicamente con interi, booleani e stringhe):
    - una per l'insieme vuoto (di un certo tipo) 
    - una per l'insieme singoletto contenente un dato elemento (di un certo tipo) 
    - e una per l'insieme contenente una data sequenza di elementi distinti (tutti di un certo tipo)
    
    __NOTA:__ Ogni insieme, anche se vuoto, mantiene sempre l'informazione sul tipo di elementi che può contenere. Quindi le espressioni `{int:}` e `{string:}` dovranno essere di tipi diversi...


2. Un **operatore unario** su insiemi:
    - `isEmpty`, che restituisca true solo se l'insieme è vuoto


3. **Operatori fra insiemi ed elementi**:
    - `Insert`, tale che `insert x A` resituisca $A \cup \{x\}$
    - `Remove`, tale che `remove x A` restituisca $A \setminus \{x\}$
    - `Member`, tale che `member x A` restituisca `true` sse $x \in A$


4. **Operatori binari** su insiemi:
    - `Union`, tale che `union A B` restituisca $A \cup B$
    - `Intersection`, tale che `intersection A B` restituisca $A \cap B$
    - `Subset`, tale che `subset A B` restituisca `true` sse $A \subset B$

In [ ]:
(* Identificatori *)
type ide = string;;

type tsub =  TSInt | TSBool | TSString;;

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound | TSet of tsub;;

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* costruttori di insiemi *)
    | Empty of tsub
    | Singleton of tsub*exp
    | Of of tsub*exp list
    (*operatori*)
    |IsEmpty of exp
    |Insert of exp*exp
    |Remove of exp*exp
    |Member of exp*exp
    |Union of exp*exp
    |Intersection of exp*exp 
    |IsSubset of exp*exp 

### Es 2: Tipi esprimibili

Estendere i tipi esprimibili e la funzione `getType` per comprendere anche i valori insieme.

In [ ]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | Set of tsub*(evT list)
    | UnBound

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | Set(t, _) -> TSet(t)
    | UnBound -> TUnBound

### Es 3: Typechecking

Estendere la funzione di typechecking in maniera tale che `typecheck(TSet, v)` restituisca `true` se e solo se `v` è un valore di tipo `TSet`, cioè è un insieme di valori, e tutti gli elementi contenuti in `v` sono del tipo adeguato.

NOTA: Può essere utilescrivere una funzione ```upcast: tsub -> tname```.

In [ ]:
(* Errori a runtime *)
exception RuntimeError of string

(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    | TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )
    |TSet(t) -> (match y with
                | Set(t', list) -> (*controllare che t' corretto e list uniforme*)
                | _ -> false)
                

### Utilities

Seguono alcune utilities sulle liste, già implementate.

In [ ]:
(* UTILITIES per le primitive del linguaggio (NON sono primitive!) *)

(* Controlla se una lista contiene o meno l'elemento dato *)
let rec list_contains l x = match l with
    | [] -> false
    | p::q -> if x = p then true else list_contains q x

(* Controlla che una lista non abbia elementi ripetuti *)
let rec checkNotEquals l = match l with
    | [] -> true
    | p::q -> if (not(list_contains q p)) then checkNotEquals q else false 

(* Requires: l1 ed l2 non hanno elementi ripetuti al loro interno
Controlla se tutti gli elementi di l1 sono contenuti in l2 o meno *)
let rec is_contained l1 l2 = match l1 with
    | [] -> true
    | p::q -> if (list_contains l2 p) then (is_contained q l2) else false

(* Rimuove l'elemento x dalla lista l se presente, dà errore altrimenti *)
let rec list_remove l x = match l with
    | [] -> raise ( RuntimeError "")
    | p::q -> if (p = x) then q else p::(list_remove q x)

### Es 4: Primitive

Implementare le primitive per gli insiemi, quali
- Creazione
- Inserzione, rimozione e appartenenza
- ```is_subset``` e ```is_empty```
- Unione intersezione

In [ ]:
(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")
    
(*###############################*)
(* Crea un nuovo insieme vuoto *)
let new_empty (t : tsub) = 

(* Crea un nuovo insieme con un solo elemento *)
let new_singleton ((t, e) : (tsub*evT)) =

(* Crea un nuovo insieme partendo da una lista di elementi. *)
let new_of ((t , l) : (tsub*(evT list))) = 



(* Verifica se un insieme contiene un elemento dato *)
let set_contains(s, v) = )

(* Inserisce un elemento in un insieme *)
let set_insert(s, v) = 

(* Rimuove un elemento da un insieme *)
let set_remove(s, v) = 




(* Verifica se un insieme è vuoto *)
let is_empty (s : evT) = 

(* Verifica se un insieme è sottoinsieme di un altro insieme *)
let set_is_subset (s1, s2) = 




let set_intersection(s1, s2) = 

let set_union(s1, s2) = 

### Es 6: Interprete

Completare la funzione eval, facendo pattern matching dei nuovi casi definiti dentro exp, e associandoli alle primitive definite precedentemente.

In [ ]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )  

In [ ]:
let e1 = Of(TString, [EString("Hello"); EString("World"); EString("from Minicaml"); ]);;
let e2 = Of(TString, [EString("Hello"); EString("World");]);;
let e3 = Let("s", Intersection(e1, e2), Contains(Den("s"), EString("Hello")));;

eval e3 emptyenv;;